In [1]:
import os
import pandas as pd
import re
import sqlite3
from hashlib import blake2b

### Prepare Data

In [2]:
# load data
df = pd.read_csv("../data/raw/PPR-ALL.csv", encoding='latin-1')
# create uuid
df = df.rename(columns={
    'Date of Sale (dd/mm/yyyy)': 'date',
    'Address': 'address',
    'Postal Code': 'postcode',
    'County': 'county',
    'Price (\x80)': 'price',
    'Not Full Market Price': 'not_full_market_price',
    'VAT Exclusive': 'vat_exclusive',
    'Property Size Description': 'floor_area',
    'Description of Property': 'construction',
})

# create uuid for each
for row in df.copy().itertuples():
    # hash_id_string = f"{row.date}{row.address}{row.price}"
    hash_id_string = f"{row.date}{row.address}{row.postcode}{row.county}{row.price}{row.not_full_market_price}{row.vat_exclusive}{row.floor_area}{row.construction}"
    df.at[row.Index, 'uuid'] = blake2b(bytes(hash_id_string, "utf-8"), digest_size=5).hexdigest()

# remove duplicates
df.drop_duplicates(inplace=True)

df.set_index('uuid', inplace=True)


/Users/br20069521/Desktop/Date-Dump/21-03-07/code_scratchpad/blog/parsing-the-property-price-register-part-1/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
db_url = "../data/database.db"
conn = sqlite3.connect(db_url)
cur = conn.cursor()
df.to_sql('ppr_all', conn)
cur.close()
conn.close()

### Create tempoary data table

In [13]:
conn = sqlite3.connect('../data/database.db')
df = pd.read_sql_query('SELECT * FROM ppr_all', conn)
df.set_index('uuid', inplace=True)

df = df.drop(["date","price","not_full_market_price","construction","floor_area","vat_exclusive"], axis=1)
df.insert(3, 'ex_eircode', None)
df.insert(3, 'ex_county', None)
df.insert(3, 'ex_postcode', None)
df.insert(3, 'address_mod', None)
df.address_mod = df.address.str.lower()


cur = conn.cursor()
cur.execute(f"DROP TABLE IF EXISTS temp;")
df.to_sql('temp', conn)
cur.close()
conn.close()

In [14]:
df

,address,postcode,county,address_mod,ex_postcode,ex_county,ex_eircode
uuid,,,,,,,
b306a21836,"5 Braemor Drive, Churchtown, Co.Dublin",None,Dublin,"5 braemor drive, churchtown, co.dublin",None,None,None
42e8eed8ad,"134 Ashewood Walk, Summerhill Lane, Portlaoise",None,Laois,"134 ashewood walk, summerhill lane, portlaoise",None,None,None
7b08b5e03d,"1 Meadow Avenue, Dundrum, Dublin 14",None,Dublin,"1 meadow avenue, dundrum, dublin 14",None,None,None
2352e180f5,"1 The Haven, Mornington",None,Meath,"1 the haven, mornington",None,None,None
cbac7ce7bd,"11 Melville Heights, Kilkenny",None,Kilkenny,"11 melville heights, kilkenny",None,None,None
...,...,...,...,...,...,...,...
b39bcc2471,"CROSSNEEN, GRAIGUECULLEN, CARLOW",None,Laois,"crossneen, graiguecullen, carlow",None,None,None
b65b929588,"GORTADERRA, SCARRIFF, CLARE",None,Clare,"gortaderra, scarriff, clare",None,None,None
3e02f239ba,"HALSEYRATH, DUNCORMICK, WEXFORD",None,Wexford,"halseyrath, duncormick, wexford",None,None,None


---

In [11]:
eircode_routing_keys = [
    "A41","A42","A45","A63","A67","A75","A81","A82","A83","A84","A85","A86","A91","A92","A94","A96","A98",
    "C15","D01","D02","D03","D04","D05","D06","D6W","D07","D08","D09","D10","D11","D12","D13","D14","D15",
    "D16","D17","D18","D20","D22","D24","E21","E25","E32","E34","E41","E45","E53","E91","F12","F23","F26",
    "F28","F31","F35","F42","F45","F52","F56","F91","F92","F93","F94","H12","H14","H16","H18","H23","H53",
    "H54","H62","H65","H71","H91","K32","K34","K36","K45","K56","K67","K78","N37","N39","N41","N91","P12",
    "P14","P17","P24","P25","P31","P32","P36","P43","P47","P51","P56","P61","P67","P72","P75","P81","P85",
    "R14","R21","R32","R35","R42","R45","R51","R56","R93","R95","T12","T23","T34","T45","T56","V14","V15",
    "V23","V31","V35","V42","V92","V93","V94","V95","W12","W23","W34","W91","X35","X42","X91","Y14","Y21",
    "Y25","Y34","Y35"]

In [12]:
regex_query = r"\b("+'|'.join(eircode_routing_keys) + r") ?[a-z0-9]{4}$"


In [ ]:
conn = sqlite3.connect('../data/database.db')
df = pd.read_sql_query('SELECT * FROM temp', conn)

In [17]:
230*230*230*230*139

388978990000

In [ ]:
388,978,990,000

In [ ]:
2,798,410,000